## Change 2

1. Can you clarify what the model used previously when AmericanObsHoursPerDay ≠ 0?
2. When AmericanObsHoursPerDay = 0, how does the PDE solver handle time discretization? Could using actual time increase numerical instability in the PDE solver (e.g., irregular grid spacing)?
3. Does this change apply only to American (continuously monitored) barriers, or also to European-style (discretely monitored) barriers?
4. Does the actual observation hour setting now align with market trading hours across exchanges? How does the model handle different markets (e.g., US vs. Asia)?
5. What happens if a product is set with AmericanObsHoursPerDay = 0, but its calendar time to maturity is less than 1 day? Is that handled properly?
6. Once americanObsDt is computed, can you explain how it is fed into the PDE grid or ctsBarrierHitPct calculation? Is it treated as the total monitoring time?
7. Previously, was a fixed dt used for every barrier monitoring interval? Does the new implementation allow varying dt that reflects actual elapsed time (e.g., 3 days vs. 1 day)?
8. Have you compared the new results against analytical barrier models or market prices to confirm that using actual time improves accuracy? If so, please provide us the test. 

## Change 4

1. Can you confirm the theoretical basis for using the formula:
=POWER(1 + linear yield × term, 1 / term) - 1
as the flat equivalent compounding yield?

📌 Purpose: Ensure MD can justify the transformation from linear to compound terms.

2. Does pricing of embedded Bermudan swaption correctly handle accreting notional?
3. Is this approach valid only for linearly accreting notional with fixed yield?
How does the model behave if:

Notional grows non-linearly?

Accretion changes mid-life?

Call dates are not annual?

4. What threshold is used to judge “materiality” in the impact test? For instance, is a PV difference of $251k on $500M trade considered material?

In the upcoming release, MD will update the model so that when Product.AmericanObsHoursPerDay is set to zero, the PDE pricing method will use the actual time interval to compute the probability of crossing the barrier. For American (continuously monitored) barriers, this parameter determines the number of hours per day during which the barrier is observed (e.g., 7.5 hours per day).

To verify the impact of the updated parameter, MD used a sample trade in the attached spreadsheet DoubleBarrierVetting.xlsm to demonstrate how the model behaves under different observation settings. Three configurations were tested—24 hours, 24 hours including weekends, and the new "Actual" setting—each producing different barrier observation intervals.

For each configuration, the corresponding time step used by the PDE engine is shown via debugging screenshots. The observation window from May 30, 2025, to June 2, 2025, spans a weekend. Under the 24-hour setting, the model computes a time step of approximately 1/365 (~0.00274). With the 24-hour plus weekend setting, the time step increases to around 1.4/365 (~0.00384). Under the new "Actual" setting, the full 3-day interval is reflected, resulting in a time step of approximately 3/365 (~0.00822).

This behavior confirms that the model correctly captures the weekend effect under the new configuration and validates that the implementation aligns with expectations.










MRM asked MD whether using settings like "24Hr", "24HrPlusWeekend", or the "Actual" 3-day interval for AmericanObsHoursPerDay makes sense for an equity underlier, given that equity markets do not trade continuously over 24 hours or during weekends. MRM also questioned whether such settings imply that the underlier is assumed to evolve continuously across non-trading hours and weekends, and requested an example if that were the case.

In response, MD explained that this behavior is primarily tied to how implied volatility is calibrated. Specifically, if the total variance is given by 
𝜎
2
𝑇
σ 
2
 T, then the discretized sum 
∑
𝜎
2
⋅
Δ
𝑡
∑σ 
2
 ⋅Δt must yield the same result. In other words, the model adjusts the diffusion by ensuring that 
𝜎
2
⋅
Δ
𝑡
σ 
2
 ⋅Δt correctly reflects the total variance over the relevant time window, regardless of whether that window includes non-trading periods.










MRM asked MD whether the change applies equally to American (continuously monitored) barriers and European-style (discretely monitored) barriers. MD responded that the change does not apply to European monitoring, as it is based on end-of-day (EOD) fixings and does not require the computation of intraday barrier crossing probabilities.

MRM asked MD whether the "actual" observation hour setting now aligns with the trading hours of different exchanges, such as those in the US versus Asia, and how the model handles such market-specific differences. MD replied that the setting is not tied to calendar-based trading hours. Instead, it relates to how implied volatility is calibrated—specifically, ensuring that 
𝜎
2
⋅
Δ
𝑡
σ 
2
 ⋅Δt is consistent with observed option prices. The model does not adjust for specific market hours but rather focuses on maintaining consistency in total variance used for pricing.

MRM asked MD what happens if a product is configured with Product.AmericanObsHoursPerDay = 0, but its calendar time to maturity is less than one day. MD replied that if the question pertains to intraday pricing, it is governed by the DayFracAddToTime parameter. For example, a value of 0.5 would represent pricing halfway through the day. This setting is also treated as an actual time interval in the model.

MRM queried MD whether americanObsDt is treated as the total monitoring time. MD replied that it is— for example, a value of 6.5 indicates that between two adjacent schedule dates, the barrier crossing is monitored for a total of 6.5 hours.

MRM asked MD what problem the new parameter is intended to solve. MD replied that, similar to a previous explanation, the purpose is to ensure that 
𝜎
2
⋅
Δ
𝑡
σ 
2
 ⋅Δt remains consistent with observed option quotes. MD also referenced the attached test results, which include impacts on present value, Greeks, and debugging screenshots as supporting evidence.